<header>
   <p  style='font-size:36px;font-family:Arial;color:#F0F0F0; background-color: #00233c; padding-left: 20pt; padding-top: 20pt;padding-bottom: 10pt; padding-right: 20pt;'>
       Anomaly Detection in Banking - Finding Outstanding Amounts using Feature Store
  <br>
       <img id="teradata-logo" src="https://storage.googleapis.com/clearscape_analytics_demo_data/DEMO_Logo/teradata.svg" alt="Teradata" style="width: 125px; height: auto; margin-top: 20pt;">
    </p>
</header>

<p style = 'font-size:20px;font-family:Arial;color:#00233C'><b>This is a Beta Release of this notebook!</b></p>   
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>   
This should be considered a Beta Release.  Every effort has been made to ensure it's functionality and completeness.  If you discover any issue with any part of this notebook, your suggestions will be reviewed and, if approved, merged in the next round of notebook commits.   
<br>   
<p style = 'font-size:20px;font-family:Arial;color:#00233C'><b>How to submit a Review of this Notebook</b></p>   
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>   
First, please read through and execute the notebook.   
<br> 
If you think the notebook is fine as it is, great.  Please send us a quick email and let us know. When the notebook is taken out of beta, this paragraph will be removed and the Beta designation the index page will be removed.   
<br> 
If you find something that you question or you think needs to be changed or if you think it's great as it is:   
</p>       
<ol style = 'font-size:16px;font-family:Arial;color:#00233C'>   
  <li>Send us an email: <a href="mailto:ClearScapeAnalyticsNotebookReviews@Teradata.com?subject=Beta Notebook Review">Click here.</a></li>   
  <li>Please include the name of the notebook in email.</li>   
  <li>Include a screen shot or a clear description of the section in the notebook you would like us to look at.</li>   
  <li>Paste the screen shot or enter the description of the section into the email.</li>   
  <li>Describe how you would change it.</li>   
  <li>Send It!</li>   
</ol>   
</p>   
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>If we have any follow-up questions, we'll reach out to you at the return email address. </p>   
<hr style="height:4px;border:none;background-color:#00233C;"> 

<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>Introduction</b></p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>In banking, tracking customer balances is vital, with anomaly detection crucial for spotting errors or fraud. This method targets anomalies in monthly balances.</p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Time and Value are critical in today’s business, with ML model development and deployment increasingly challenging as prediction becomes central to competitiveness.</p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Successful AI/ML implementations face three main challenges:</p>
<li style = 'font-size:16px;font-family:Arial;color:#00233C'><b>The Data Problem:</b> Quality data and feature engineering consume 80% of the implementation time. Even when different use cases share the same source data and features, organizations often handle data preparation separately.</li>
<li style = 'font-size:16px;font-family:Arial;color:#00233C'><b>The Scale Problem:</b> Real-world use cases often require multiple models. In production, these models require fresh features engineered in the same way as during training. Ensuring the auditability of these features behind model decisions is crucial.</li>
<li style = 'font-size:16px;font-family:Arial;color:#00233C'><b>The Deployment Problem:</b> Transitioning prototypes to production, especially operationalizing data prep pipelines, is often problematic.</li></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Addressing these challenges requires strategic planning, skilled talent, and integration with existing systems. Oraganizations with a history in Data Management recognize the benefits of reusable Data Products, making Enterprise Feature Stores a valuable investment.</p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>A Feature Store is a curated repository of pre-calculated features, simplifying the journey from data to actionable insights. An Enterprise Feature Store extends across domains/teams, incorporating a Governance Framework for predictable feature delivery. </p>
    
<p style = 'font-size:16px;font-family:Arial;color:#00233C'><b>While most features are reusable, some need model-specific calculations before integration into a unified dataset.</b></p>
    
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>The key difference between Feature Store (FS) and Enterprise Feature Store (EFS) is the scope across multiple domains/teams along with the Governance Framework (that gives an assurance that features are delivered under predictable SLAs and it also defines the operating model how the EFS is used across different teams/domains and how features lifecycle is managed). Although most Features are considered as re-usable, there is still some minor part of Features that must be calculated as model-specific (e.g., scaled variables, principal components, etc.) and then combined with the rest of the pre-calculated Features into a single data set (ADS). The figure below describes this co-existence of model-specific ADS(es) and model-independent EFS.</p>

<img src='images/EFS.png'>


<p style = 'font-size:16px;font-family:Arial;color:#00233C'><b><i>This use case provides insight into real customer needs for Feature Store functionalities, derived from a large French bank's ongoing project.</i></b></p>

<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>Business Values</b></p>

<li style = 'font-size:16px;font-family:Arial;color:#00233C'>Rapid model creation and deployment through enterprise feature reuse.</li>
<li style = 'font-size:16px;font-family:Arial;color:#00233C'>Flexible creation and usage of new features without extensive engineering support.</li>
<li style = 'font-size:16px;font-family:Arial;color:#00233C'>Consistent definitions ensure accuracy and quick deployment.</li>
<li style = 'font-size:16px;font-family:Arial;color:#00233C'>Collaboration and sharing of features among teams.</li>
<li style = 'font-size:16px;font-family:Arial;color:#00233C'>Maintained feature lifecycle for compliance and auditability.</li>
</p>

<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>Why Vantage?</b></p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>There are several reasons why EFS naturally fits to Teradata Vantage:</p>
<li style = 'font-size:16px;font-family:Arial;color:#00233C'>Utilizes Teradata Vantage with its powerful Analytical Library and SQL Engine.</li>
<li style = 'font-size:16px;font-family:Arial;color:#00233C'>Primary Index enables efficient single-row access for online feature use.</li>
<li style = 'font-size:16px;font-family:Arial;color:#00233C'>Single platform for both online and offline feature stores.</li>
<li style = 'font-size:16px;font-family:Arial;color:#00233C'>Macros reduce parsing overhead from API access.</li>
<li style = 'font-size:16px;font-family:Arial;color:#00233C'>R and Python code execution within SQL Engine.</li>
<li style = 'font-size:16px;font-family:Arial;color:#00233C'>Bi-temporal querying capability.</li>
<li style = 'font-size:16px;font-family:Arial;color:#00233C'>Scalable MPP power for feature computation.</li>
<li style = 'font-size:16px;font-family:Arial;color:#00233C'>Industry-specific Logical Data Model as a feature source.</li>
<li style = 'font-size:16px;font-family:Arial;color:#00233C'>Connectivity to Object Storage via NOS for feature data sourcing.</li>
<li style = 'font-size:16px;font-family:Arial;color:#00233C'>Query Grid facilitates access to multiple data sources.</li>
<li style = 'font-size:16px;font-family:Arial;color:#00233C'>Close-to-real-time feature delivery using Query Services and Teradata Intelligent Memory.</li>
<li style = 'font-size:16px;font-family:Arial;color:#00233C'>Workload management prioritizes tasks effectively.</li></p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>The unique massively-parallel architecture of Teradata Vantage allows users to prepare data, train, evaluate, and deploy models at unprecedented scale.</p>


<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>Methodology</b></p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>In this demo we have used a methodology which involves analyzing a time series of data, where each data point represents the outstanding amount at the end of each month. To detect anomalies, we use the following steps:</p>

<li style = 'font-size:16px;font-family:Arial;color:#00233C'><b>Model the Distribution:</b> We assume that the historical data of monthly balances follow a normal distribution. This distribution is characterized by two parameters: the mean (μ) and the standard deviation (σ). These are the features of the Entity </li>

<li style = 'font-size:16px;font-family:Arial;color:#00233C'><b>Compute the Z-Score:</b> For the most recent monthly balance (the latest data point in the time series), we compute its Z-score. The Z-score is a statistical measure that describes a value's relationship to the mean of a group of values. It is calculated using the formula: </li>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'><div><code>Z = (X - μ) / σ</code></div></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>where X is the value in question, μ is the mean, and σ is the standard deviation.</p>

<li style = 'font-size:16px;font-family:Arial;color:#00233C'><b>Threshold for Anomaly Detection:</b> We set a threshold for the Z-score. If the absolute value of the Z-score for the latest monthly balance exceeds this threshold, it is flagged as an anomaly.</li></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>It's important to note that the computation of the Z-score and the anomaly flag is dependent on the values of the mean and standard deviation. These dependent features are not computed at the same time as the static features but are derived later, once the latest outstanding amount (the new data point) becomes available.</p>

<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>Feature Engineering</b></p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Feature engineering is a crucial step in the entity-feature paradigm, as it involves creating and transforming features to better represent the underlying problem for predictive modeling. In our case, the feature engineering process is twofold, each with its specific inputs and outputs. Below are the processes that are a part of this feature engineering</p>

<li style = 'font-size:16px;font-family:Arial;color:#00233C'><b>Process 1:</b> Computing Mean and Standard Deviation</li>
<li style = 'font-size:16px;font-family:Arial;color:#00233C'><b>Process 2:</b> Computing Z-Score and Anomaly Flag</li>
<li style = 'font-size:16px;font-family:Arial;color:#00233C'><b>Roll Out:</b> Feature Engineering rollout
    <ul style = 'font-size:16px;font-family:Arial;color:#00233C'>
    <li>Addressing Circular Dependency</li>
    <li>Roll out after adjusting circular dependency</li></ul>
<li style = 'font-size:16px;font-family:Arial;color:#00233C'><b>Validation:</b> Feature Store Validation</li>
</p>

<hr style="height:2px;border:none;background-color:#00233C;">

<p style = 'font-size:20px;font-family:Arial;color:#00233C'><b>1. Connect to Vantage.</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>In the section, we import the required libraries and set environment variables and environment paths (if required).</p>

In [ ]:
%%capture
# # '%%capture' suppresses the display of installation steps of the following packages
!pip install tdfs4ds==0.2.2.5

<div class="alert alert-block alert-info">
<p style = 'font-size:16px;font-family:Arial;color:#00233C'><b>Note: </b><i>The above statements may need to be uncommented if you run the notebooks on a platform other than ClearScape Analytics Experience that does not have the libraries installed. If you uncomment those installs, be sure to restart the kernel after executing those lines to bring the installed libraries into memory. The simplest way to restart the Kernel is by typing zero zero: <b> 0 0</b></i></p>
</div>

In [ ]:
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter(action='ignore', category=DeprecationWarning)
warnings.simplefilter(action='ignore', category=RuntimeWarning)
warnings.simplefilter(action='ignore', category=FutureWarning)

import json
import getpass
import pandas as pd
import datetime
from teradataml import *
import numpy as np


display.max_rows = 5

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>We will be prompted to provide the password. We will enter the password, press the Enter key, and then use the down arrow to go to the next cell. Begin running steps with Shift + Enter keys.</p>

In [ ]:
%run -i ../startup.ipynb
eng = create_context(host = 'host.docker.internal', username='demo_user', password = password)
print(eng)

In [ ]:
%%capture
execute_sql('''SET query_band='DEMO=AnomalyDetection_OutstandingAmount_EFS.ipynb;' UPDATE FOR SESSION; ''')

<hr style="height:2px;border:none;background-color:#00233C;">

<p style = 'font-size:20px;font-family:Arial;color:#00233C'><b>2. Getting Data for This Demo</b></p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>We have provided data for this demo on cloud storage. We have the option of either running the demo using foreign tables to access the data without using any storage on our environment or downloading the data to local storage, which may yield somewhat faster execution. However, we need to consider available storage. There are two statements in the following cell, and one is commented out. We may switch which mode we choose by changing the comment string.</p>   


In [ ]:
# %run -i ../run_procedure.py "call get_data('DEMO_AnomalyDetection_EFS_cloud');"
 # Takes about 20 seconds
%run -i ../run_procedure.py "call get_data('DEMO_AnomalyDetection_EFS_local');"
 # Takes about 30 secs

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Optional step – We should execute the below step only if we want to see the status of databases/tables created and space used.</p>

In [ ]:
%run -i ../run_procedure.py "call space_report();"

<hr style="height:2px;border:none;background-color:#00233C;">

<p style = 'font-size:20px;font-family:Arial;color:#00233C'><b>3. Analyze the raw data set</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Let us start by creating a teradataml dataframe. A "Virtual DataFrame" that points directly to the dataset in Vantage.</p>



In [ ]:
source_data = DataFrame(in_schema("DEMO_AnomalyDetection_EFS","OUTSTANDING_AMOUNTS"))
source_data

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>We will create the list of dates that are existing in the source data</p>

In [ ]:
list_Balance_Dates = sorted([str(d[0]) for d in source_data.groupby('Balance_Date').count().get('Balance_Date').get_values()])

<hr style="height:2px;border:none;background-color:#00233C;">

<p style = 'font-size:20px;font-family:Arial;color:#00233C'><b>4. Setup the feature store</b></p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>We will start by setting up the database used for the creation of feature store</p>

In [ ]:
Param = {
    'database': 'demo_user',
    'temp_database_name': 'demo_user'
}    
print(Param)
import tdfs4ds
tdfs4ds.setup(database = Param['database'])

In [ ]:
from tdfs4ds.feature_store.entity_management import remove_entity
remove_entity(entity='Account_ID',data_domain='OUTSTANDING_AMOUNTS')

<hr style="height:1px;border:none;background-color:#00233C;">

<p style = 'font-size:20px;font-family:Arial;color:#00233C'><b>4.1 Managing the time</b></p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'><b>Build a TimeManager</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>The TimeManager function from the utils package of the tdfs4ds library initializes the TimeManager with a table name, schema name, and optionally a data type. If the table doesn't exist, it creates one with a BUSINESS_DATE column of the specified data type.</p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>The various functions available in the TimeManager are: </p>
<li style = 'font-size:16px;font-family:Arial;color:#00233C'><code>display:</code> Displays the table.</li>
<li style = 'font-size:16px;font-family:Arial;color:#00233C'><code>update:</code> Updates the BUSINESS_DATE.</li>
<li style = 'font-size:16px;font-family:Arial;color:#00233C'><code>get_date_in_the_past:</code> Retrieves the earliest date and time value from the table </p>

In [ ]:
from tdfs4ds.utils.time_management import TimeManager
business_time = TimeManager(table_name = 'BUSINESS_DATE', schema_name = 'demo_user')
business_time.display()

In [ ]:
business_time.update(new_time=list_Balance_Dates[9])
business_time.display()

In [ ]:
business_time.get_date_in_the_past()

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>FEATURE_STORE_TIME is used to define a specific timestamp for querying the feature data, enabling temporal feature querying. If FEATURE_STORE_TIME is None, the query defaults to the current valid time. Here we are setting the time to the past date as we are using the past data for our feature store. The FEATURE_STORE_TIME will be used in the roll out process. In the roll out process the FEATURE_STORE_TIME is set based on the list of dates on which the roll out function iterates.</p>

In [ ]:
tdfs4ds.FEATURE_STORE_TIME = business_time.get_date_in_the_past()

<p style = 'font-size:16px;font-family:Arial;color:#00233C'><b>Use the TimeManager to filter out the source data</b></p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Here we will select dates only for the past i.e all the data where the Balance Date is less than or equal to Business Date</p>

In [ ]:
df_business_date = DataFrame('BUSINESS_DATE')
df_business_date

In [ ]:
input_source = source_data.merge(right=df_business_date, how='inner', on=[source_data.Balance_Date<=df_business_date.BUSINESS_DATE])
input_source

In [ ]:
input_source.shape

<hr style="height:2px;border:none;background-color:#00233C;">

<p style = 'font-size:20px;font-family:Arial;color:#00233C'><b>5. Feature Engineering - Initialization</b></p>

<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>5.1 Process 0 : Count the number of Monthly Balance per Account</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>In this step we will do manual engineering using teradataml</p>

In [ ]:
df_nb_points_dev = input_source.groupby('Account_ID').count()
df_nb_points_dev

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>We will save this process as a permanent view using the crystallize_view function.</p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'> the crystallize_view function crystallizes a TeradataML (tdml) DataFrame into a view with specified name and schema. This function takes a TeradataML DataFrame (tddf), generates a dependency graph for its SQL representation, and replaces or creates sub-views with new names in the database. It then constructs the final view by replacing old names with new ones in the SQL representation and executes the query to create the main view. The result is returned as a TeradataML DataFrame representing the created view.</p>

In [ ]:
from tdfs4ds.utils.lineage import crystallize_view
df_nb_points = crystallize_view(df_nb_points_dev,view_name='PROCESS_0',schema_name='demo_user')
df_nb_points

<p style = 'font-size:16px;font-family:Arial;color:#00233C'><b>Register and Upload Features in feature store </b></p> 
    
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>The upload_features register the feature engineering process implemented with the teradata dataframe in the process catalog (and attribute a process id), registers the entity and the features in the feature catalog, performs the feature engineering process at the FEATURE_STORE_TIME and ingests the calculated features in the feature store data model.</p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Required Parameters</p>
<li style = 'font-size:16px;font-family:Arial;color:#00233C'><code>df (tdml.DataFrame):</code> The input DataFrame containing the feature data.</li>
<li style = 'font-size:16px;font-family:Arial;color:#00233C'><code>entity_id (list/dict/other):</code> A representation of the entity ID. If a dictionary, the keys are used as entity IDs. If a list or another data type, it is used directly as the entity ID.</li>
<li style = 'font-size:16px;font-family:Arial;color:#00233C'><code>feature_names (list):</code> A list of column name corresponding to the features you want to ingest in the feature store. 
<li style = 'font-size:16px;font-family:Arial;color:#00233C'><code>**kwargs:</code> Additional keyword arguments that can be passed to the function. The metadata argument is used to document the features.</li></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'> The function returns a teradataml dataframe that retrieves the calculated features read from the feature store data model, providing fast access to calculated features.</p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>First we define the variables to be used</p>
<li style = 'font-size:16px;font-family:Arial;color:#00233C'><code>entity_id:</code>dict or compatible type. The entity identifier. If not a dictionary, it will be converted using get_column_types.</li>
<li style = 'font-size:16px;font-family:Arial;color:#00233C'><code>feature_names:</code> list. The list of feature names to be uploaded.</li>
<li style = 'font-size:16px;font-family:Arial;color:#00233C'><code>metadata:</code> dict, optional. Additional metadata to associate with the upload. Defaults to an empty dictionary.</li></p>

In [ ]:
# Specify the entity ids in the view columns (+ data types) 
entity_id       = ['Account_ID']
# Specify the columns that contains the actual features
feature_names   = ['COUNT_MONTHLY_BALANCE']
# attach informative metadata to document your process
metadata        = {'project' : 'test'}

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>DATA_DOMAIN is the data domain under which the table and view will be categorized. If not specified, the function uses a default value which is the default database when specified in the context and when the tdfs4ds is imported after the connection.. Here we set the value of the DATA_DOMAIN to OUTSTANDING_AMOUNTS.</p>

In [ ]:
tdfs4ds.DATA_DOMAIN = 'OUTSTANDING_AMOUNTS'

In [ ]:
from tdfs4ds import upload_features
dataset = upload_features(
    df=df_nb_points,
    entity_id=entity_id,
    feature_names=feature_names,
    metadata = metadata
)

In [ ]:
dataset

<hr style="height:1px;border:none;background-color:#00233C;">

<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>5.2 Process 1 : Computing Mean and Standard Deviation</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'><b>Input:</b> Historical data of monthly outstanding amounts.</p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'><b>Process:</b> This first process involves calculating two key statistical measures from the historical monthly balances: the mean (μ) and the standard deviation (σ). </p>

<li style = 'font-size:16px;font-family:Arial;color:#00233C'>The **mean** provides an average value of the historical data, serving as a central reference point.</li>
<li style = 'font-size:16px;font-family:Arial;color:#00233C'>The **standard deviation** measures the variation or dispersion of the balances from the mean, indicating the typical range of fluctuation.</li></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'><b>Output:</b>
    <li style = 'font-size:16px;font-family:Arial;color:#00233C'>Mean (μ) of historical balances</li>
<li style = 'font-size:16px;font-family:Arial;color:#00233C'>Standard Deviation (σ) of historical balances</li></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>These outputs serve as static features that describe the general behavior of the account's monthly balances over time.</p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>We will use the selected features and build a dataset with these features.</p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>The get_feature_versions function will retrieve feature versions for specified features associated with certain entities from a given data domain. This function allows fetching either all versions or just the latest versions of the features. The feature version is the process_id that computed these features.</p>

In [ ]:
from tdfs4ds.feature_store.feature_query_retrieval import get_available_features,get_feature_versions
selected_features = get_feature_versions(entity_name= entity_id, features=feature_names, domain = 'OUTSTANDING_AMOUNTS')
selected_features

<p style = 'font-size:16px;font-family:Arial;color:#00233C'><b>Build Dataset with selected features</b></p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>The build_dataset function creates a dataset view from the feature store. This dataset is a snapshot of the values of the selected features at the FEATURE_STORE_TIME. If we want the current version, i.e. the latest, we should set FEATURE_STORE_TIME to None before calling build_dataset.</p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Required Parameters</p>
<li style = 'font-size:16px;font-family:Arial;color:#00233C'><code>entity_id:</code> is the list of column names defining the entity.</li> 
<li style = 'font-size:16px;font-family:Arial;color:#00233C'><code>selected_features:</code> is the dictionary with feature names as key and feature version (meaning the process id that calculates these features) are values.
<li style = 'font-size:16px;font-family:Arial;color:#00233C'><code>view_name:</code> is the name of the dataset view we want. </li>

In [ ]:
from tdfs4ds import build_dataset
nb_points_efs = build_dataset(entity_id=entity_id,selected_features=selected_features,view_name='nb_points_efs', time_manager=business_time)
nb_points_efs = DataFrame(in_schema("demo_user","nb_points_efs"))
nb_points_efs = nb_points_efs[nb_points_efs.COUNT_MONTHLY_BALANCE > 9]
nb_points_efs

In [ ]:
from sqlalchemy import func
source_efs = input_source.merge(right=nb_points_efs, how='inner', on = 'Account_ID', lsuffix = 't1', rsuffix = 't2')
source_efs = source_efs.assign(drop_columns=True,
                               Account_ID = source_efs.Account_ID_t1,
                               Balance_Date = source_efs.Balance_Date,
                               Monthly_Balance = source_efs.Monthly_Balance)

source_efs = source_efs.select(['Account_ID','Monthly_Balance'])
source_efs

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Here, we will calculate the mean and standard deviation for the monthly balance for each account</p>

In [ ]:
mean_std_dev = source_efs.groupby('Account_ID').assign(std_monthly_balance = func.stddev_samp(source_efs.Monthly_Balance.expression)
                                                      ,mean_monthly_balance = func.avg(source_efs.Monthly_Balance.expression))
mean_std_dev

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>We will save the output of this process as a permanent view using the crystallize_view function.</p>

In [ ]:
mean_std = crystallize_view(mean_std_dev,view_name='mean_std', schema_name='demo_user')
mean_std

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>We will register these newly created features in the feature store and upload.</p>

In [ ]:
# Specify the entity ids in the view columns (+ data types) 
entity_id       = ['ACCOUNT_ID']
# Specify the columns that contains the actual features
feature_names   = ['MEAN_MONTHLY_BALANCE','STD_MONTHLY_BALANCE']
# attach informative metadata to document your process
metadata        = {'project' : 'test'}

In [ ]:
dataset = upload_features(
    df=mean_std,
    entity_id=entity_id,
    feature_names=feature_names,
    metadata = metadata
)

<hr style="height:1px;border:none;background-color:#00233C;">

<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>5.3 Process 2 : Computing Z-score and anomaly score</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'><b>Input:</b> Latest outstanding amount and the outputs from Process 1 (mean and standard deviation).</p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'><b>Process:</b> Once the latest monthly balance is available, this process computes two additional features:</p>

<li style = 'font-size:16px;font-family:Arial;color:#00233C'>The **Z-Score**, calculated as \( Z = (X - μ) / σ \), where \(X\) is the latest outstanding amount. It indicates how many standard deviations the latest balance deviates from the mean.</li>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>The **Anomaly Flag**, a binary indicator (1 or 0), is derived from the Z-score. If the absolute value of the Z-score exceeds a predefined threshold, it indicates an anomaly (flagged as 1); otherwise, it is considered normal (flagged as 0).</li>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'><b>Output:</b></p>
<li style = 'font-size:16px;font-family:Arial;color:#00233C'>Z-Score of the latest outstanding amount</li>
<li style = 'font-size:16px;font-family:Arial;color:#00233C'>Anomaly Flag indicating the presence of an anomaly</li></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>These dynamic, dependent features are crucial for identifying significant deviations in the latest outstanding amount, enabling timely anomaly detection.</p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>We will fetch the uploaded features and than build the dataset using these features</p>

In [ ]:
selected_features = get_feature_versions(entity_name= entity_id, features=feature_names, domain = 'OUTSTANDING_AMOUNTS')
selected_features

In [ ]:
mean_std_efs = build_dataset(entity_id=entity_id,selected_features=selected_features,view_name='mean_std_efs', time_manager=business_time)

In [ ]:
mean_std_efs = DataFrame(in_schema('demo_user','mean_std_efs'))
mean_std_efs

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>We will use the dataset where the balance date is equal to the BUSINESS_DATE and use that dataset to calculate the Z_SCORE and the ANOMALY based on the Z_SCORE</p>

In [ ]:
source_data = DataFrame(in_schema("DEMO_AnomalyDetection_EFS","OUTSTANDING_AMOUNTS"))
df_business_date = DataFrame(in_schema("demo_user","BUSINESS_DATE"))

In [ ]:
input_zscore_source = source_data.merge(right=df_business_date, how='inner', on=[source_data.Balance_Date==df_business_date.BUSINESS_DATE])
input_zscore_source

In [ ]:
# input_zscore_source = source_data[source_data.Balance_Date == business_date]
source_zscore = input_zscore_source.merge(right=mean_std_efs, how='inner', on = ["ACCOUNT_ID = Account_ID"], lsuffix = 't1', rsuffix = 't2')
source_zscore = source_zscore.assign(drop_columns=True,
                               Account_ID = source_zscore.ACCOUNT_ID_t1,
                               Z_SCORE = (source_zscore.Monthly_Balance-source_zscore.MEAN_MONTHLY_BALANCE)
                                                       /source_zscore.STD_MONTHLY_BALANCE.nullifzero())
anomaly_dev = source_zscore.assign(ANOMALY_FLAG = case([(source_zscore.Z_SCORE.abs() > 3, 1)], else_ = 0) )
anomaly_dev

In [ ]:
anomaly_dev.groupby('ANOMALY_FLAG').count()

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>We will save the process as permanent view</p>

In [ ]:
anomaly = crystallize_view(anomaly_dev,view_name='anomaly', schema_name='demo_user')

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>We will register in the feature store</p>

In [ ]:
anomaly= DataFrame(in_schema('demo_user','anomaly'))
anomaly

In [ ]:
# Specify the entity ids in the view columns (+ data types) 
entity_id       = ['ACCOUNT_ID']
# Specify the columns that contains the actual features
feature_names   = ['Z_SCORE','ANOMALY_FLAG']
# attach informative metadata to document your process
metadata        = {'project' : 'test'}

In [ ]:
dataset = upload_features(
    df=anomaly,
    entity_id=entity_id,
    feature_names=feature_names,
    metadata = metadata
)

<hr style="height:2px;border:none;background-color:#00233C;">

<p style = 'font-size:20px;font-family:Arial;color:#00233C'><b>6. Feature Engineering - Roll out</b></p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>We will do a refinement of the Feature Engineering Process. In the context of anomaly detection for outstanding amounts in the banking sector, a refinement is needed in the feature engineering process to address a potential circular dependency. The key modification is to exclude data points identified as anomalies in past computations when calculating the static features (mean and standard deviation).</p>

<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>6.1 Addressing Circular Dependency</b></p>
    
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>The original methodology has a circular dependency:</p>

<li style = 'font-size:16px;font-family:Arial;color:#00233C'><b>Process 1</b> computes the mean and standard deviation, which are used as inputs for Process 2.</li>
    <li style = 'font-size:16px;font-family:Arial;color:#00233C'><b>Process 2</b> computes the Z-score and anomaly flag for the latest data point.</li>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>However, if anomalies identified by **Process 2** are included in the dataset for **Process 1**, it can skew these static values and affect future anomaly detection.</p>



In [ ]:
selected_features = get_feature_versions(entity_name= entity_id, features=['ANOMALY_FLAG'], domain = 'OUTSTANDING_AMOUNTS')
selected_features

In [ ]:
from tdfs4ds.feature_store.feature_query_retrieval import list_features
list_features()

In [ ]:
source_data = DataFrame(in_schema("DEMO_AnomalyDetection_EFS","OUTSTANDING_AMOUNTS"))
df_business_date = DataFrame(in_schema("demo_user","BUSINESS_DATE"))

In [ ]:
input_source_past = source_data.merge(right=df_business_date, how='inner', on=[source_data.Balance_Date<df_business_date.BUSINESS_DATE])
input_source_past

In [ ]:
from tdfs4ds import build_dataset_time_series
mean_df = build_dataset_time_series(input_source_past, 'Balance_Date', entity_id, selected_features, 
                                    time_manager=business_time)
mean_df = mean_df.assign(mon_balance = case([(mean_df.ANOMALY_FLAG == 1, None)], else_ = mean_df.Monthly_Balance))
mean_std_dev = mean_df.groupby('Account_ID').assign(std_monthly_balance = 
                                                    func.stddev_samp(mean_df.mon_balance.expression)
                                                      ,mean_monthly_balance = 
                                                    func.avg(mean_df.mon_balance.expression))

mean_std_dev

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>We will save the modified view as a permanent view</p>

In [ ]:
mean_std = crystallize_view(mean_std_dev,view_name='mean_std', schema_name=Param['database'])

<hr style="height:1px;border:none;background-color:#00233C;">
<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>6.2 The roll out after adjusting circular dependency</b></p>
    
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>To resolve this, we adjust the methodology as follows:</p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'><b>Revised Process 1: Excluding Anomalies in Static Feature Computation</b></p>

 <li style = 'font-size:16px;font-family:Arial;color:#00233C'><b>Input:</b> Historical data of monthly outstanding amounts, excluding those identified as anomalies.</li>

  <li style = 'font-size:16px;font-family:Arial;color:#00233C'><b>Process:</b> This revised process calculates the mean and standard deviation of the historical monthly balances, but with a critical adjustment – it excludes any data points previously flagged as anomalies. By doing so, it ensures that the static features represent the 'normal' behavior of the account's monthly balances.</li>
</p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'><b>Output:</b></p> 
<li style = 'font-size:16px;font-family:Arial;color:#00233C'>Adjusted Mean (μ) of historical balances (excluding anomalies)</li>
<li style = 'font-size:16px;font-family:Arial;color:#00233C'>Adjusted Standard Deviation (σ) of historical balances (excluding anomalies)</li>
</p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'><b>Continuation with Process 2: As Previously Defined</b></p>

<li style = 'font-size:16px;font-family:Arial;color:#00233C'><b>Input:</b> Latest outstanding amount and the outputs from the revised Process 1.</li>

<li style = 'font-size:16px;font-family:Arial;color:#00233C'><b>Process and Output:</b> Remain the same as previously defined. The Z-score and anomaly flag are computed using the adjusted mean and standard deviation.</li>
</p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'><b>Implications of the Refinement</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>This refinement has significant implications:</p>

<li style = 'font-size:16px;font-family:Arial;color:#00233C'>It ensures that the static features (mean and standard deviation) are not influenced by anomalous data, which could otherwise distort the understanding of 'normal' behavior.</li>
<li style = 'font-size:16px;font-family:Arial;color:#00233C'>It adds a layer of historical awareness to the model, as the anomaly detection for a current data point depends on the refined understanding of past data.</li>
<li style = 'font-size:16px;font-family:Arial;color:#00233C'>It resolves the circular dependency, making the feature engineering process more robust and the anomaly detection more accurate.</li>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>First we get the list of processes to run sequentially</p>

In [ ]:
from tdfs4ds.process_store.process_query_administration import list_processes
list_process_ = list_processes()
list_process_

In [ ]:
from tdfs4ds.process_store.process_query_administration import get_process_id
db = Param['database']
process_0_id = get_process_id(f'"{db}"."PROCESS_0"')
process_1_id = get_process_id(f'"{db}"."mean_std"')
process_2_id = get_process_id(f'"{db}"."anomaly"')
print('process_0_id:',process_0_id)
print('process_1_id:',process_1_id)
print('process_2_id:',process_2_id)

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>We will now execute the roll out process. This process executes a series of processes for each date in a given list, managing the time and logging settings. It iterates over a list of dates, updating a TimeManager object with each date, and then executes a list of processes for that date. It also manages the synchronization of time for a feature store and disables display logs during its execution. It will also set global variables DISPLAY_LOGS and FEATURE_STORE_TIME. It catches and prints exceptions along with the date on which they occurred.</p>


<div class="alert alert-block alert-warning">
    <p style = 'font-size:16px;font-family:Arial;color:#00233C'><b><i>**Note: The roll out process will take approximately 10-15 minutes as it runs the process for 10 different dates</i></b></div>

In [ ]:
from tdfs4ds import roll_out
roll_out(
    date_list = list_Balance_Dates[10::],
    process_list = [process_0_id, process_1_id,process_2_id],
    time_manager = business_time
)

In [ ]:
business_time.get_date_in_the_past()

<hr style="height:2px;border:none;background-color:#00233C;">

<p style = 'font-size:20px;font-family:Arial;color:#00233C'><b>7. Validation of feature Store</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>The previous approach starts looking complex with a use case pretty simple. Moreover, we see that this pattern is use case or pipeline dependent, leading to a complex implementation because specific. The enterprise feature store approach aims to reduce the complexity of the implementation by splitting the pipeline in simple processes.</p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>We see that there are only two operations to consider:</p>
<li style = 'font-size:16px;font-family:Arial;color:#00233C'>the process to feature store operation</li>
<li style = 'font-size:16px;font-family:Arial;color:#00233C'>the feature store to feature set operation, with time travel enabled</li>
</p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>what remains specific to the use case is the building of the dataset, which requires the join between feature set and sources coming from the production area. Beneath the surface, we also guess the problematic of the cold start.</p>

<hr style="height:1px;border:none;background-color:#00233C;">

<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>7.1 How many anomalies have been spotted?</b></p>

In [ ]:
from tdfs4ds.utils.time_management import TimeManager
business_time = TimeManager(table_name = 'BUSINESS_DATE', schema_name = Param['database'])
df_business_date =  DataFrame('BUSINESS_DATE')


In [ ]:
source_data = DataFrame(in_schema("DEMO_AnomalyDetection_EFS","OUTSTANDING_AMOUNTS"))

input_source = source_data.merge(right=df_business_date, how='inner', on=[source_data.Balance_Date<=df_business_date.BUSINESS_DATE])
input_source

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>We will use the build_dataset_time_series function which constructs a time series dataset based on the specified features and entity_id from the provided dataframe.</p>

In [ ]:
tdfs4ds.SCHEMA = Param['database']
tdfs4ds.DATA_DOMAIN = 'OUTSTANDING_AMOUNTS'
from tdfs4ds.feature_store.feature_query_retrieval import get_feature_versions
from tdfs4ds import build_dataset_time_series

entity_id = ['Account_Id']
features  = ['MEAN_MONTHLY_BALANCE','STD_MONTHLY_BALANCE','Z_SCORE','ANOMALY_FLAG']
selected_features = get_feature_versions(entity_name=entity_id , features=features, domain = 'OUTSTANDING_AMOUNTS')
#print(selected_features)
df = build_dataset_time_series(input_source, 'Balance_Date', entity_id, selected_features, time_manager=business_time)

In [ ]:
selected_features

In [ ]:
df.groupby('ANOMALY_FLAG').count()

<hr style="height:1px;border:none;background-color:#00233C;">

<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>7.2 What are the accounts with the most of anomalies?</b></p>

In [ ]:
df[df.ANOMALY_FLAG == 1].groupby('Account_ID').count().sort('count_ANOMALY_FLAG', ascending=False)

<hr style="height:1px;border:none;background-color:#00233C;">

<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>7.3 We will take a closer look on the most suspicious account.</b></p>

<p style = 'font-size:14px;font-family:Arial;color:#00233C'><b>**Note: Please replace the account number in the cell below with the one which we would like to analyze</p>

In [ ]:
account_id = 370417

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>We can see that the required data is available to us in teradataml dataframe. Let's visualize this data to better understand the anomaly in each account. Vantage's Clearscape Analytics can easily integrate with 3rd party visualization tools like Tableau, PowerBI or many python modules available like plotly, seaborn etc. We can do all the calculations and pre-processing on Vantage and pass only the necessary information to visualization tools, this will not only make the calculation faster but also reduce the time due to less data movement between tools.</p>

In [ ]:
from plotanomalyfunction import plot_account_data
plot_account_data(df,account_id )

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>In the above graph we can see that the green dots which are in between the dotted lines which are calculated by subtracting and adding 3 times the standard deviation of Monthly balance from the mean monthly balance. The red dots that lie outside these range are considered as anomalies.</p>

<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>Conclusion</b></p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Thus, having an enterprise-curated set of Features, not only provides a decrease in the time to create and deploy models but also provides consistency in the definitions of the data. So, if a company makes its strategic decisions based on the insights it receives from the Data Warehouse and acts based on those insights. Ideally, the actions are data-driven, to affect an outcome. If the same data definitions are used to affect the outcome, as is used to gain the initial insights, then there is more likely to be aligned to the strategy, and the outcome will have a positive impact on the business.</p>

<hr style="height:2px;border:none;background-color:#00233C;">
<p style = 'font-size:20px;font-family:Arial;color:#00233C'><b>8. Cleanup</b></p>
<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>Work Tables and Views</b></p>

In [ ]:
tables = ['FS_PROCESS_CATALOG','FS_FEATURE_CATALOG','FS_T_OUTSTANDING_AMOUNTS_Account_ID_VARCHAR_LATIN',
          'tdfs__fgjnojnsmdoignmosnig','FS_T_OUTSTANDING_AMOUNTS_Account_ID_VARCHAR_UNICODE','temp',
          'BUSINESS_DATE','FS_T_OUTSTANDING_AMOUNTS_Account_ID_FLOAT','FS_T_OUTSTANDING_AMOUNTS_Account_ID_BIGINT']

# Loop through the list of tables and execute the drop table command for each table
for table in tables:
    try:
        db_drop_table(table_name=table)
    except:
        pass   

In [ ]:
views = ['anomaly_sub_1','anomaly','mean_std_sub_4','mean_std_efs','MEAN_STD_SUB_1','mean_std_sub_3','MEAN_STD_SUB_0',
         'mean_std','PROCESS_0','MEAN_STD_SUB_2','FS_V_OUTSTANDING_AMOUNTS_Account_ID_VARCHAR_UNICODE',
         'FS_V_OUTSTANDING_AMOUNTS_Account_ID_VARCHAR_LATIN','ml__aggregate_count__1711528657804036',
         'FS_V_OUTSTANDING_AMOUNTS_Account_ID_BIGINT','PROCESS_0_sub_1','PROCESS_0_SUB_0',
         'FS_V_OUTSTANDING_AMOUNTS_Account_ID_FLOAT','nb_points_efs']

# Loop through the list of tables and execute the drop table command for each table
for viewname in views:
    try:
        db_drop_view(view_name=viewname)
    except:
        pass

<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>Databases and Tables</b></p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>The following code will clean up tables and databases created above.</p>

In [ ]:
%run -i ../run_procedure.py "call remove_data('DEMO_AnomalyDetection_EFS');" 
#Takes 40 seconds

In [ ]:
remove_context()

<hr style="height:2px;border:none;background-color:#00233C;">
<b style = 'font-size:20px;font-family:Arial;color:#00233C'>Required Materials</b>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Let’s look at the elements we have available for reference for this notebook:</p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Check more details regarding the tdfs4ds package functions <a href='https://pypi.org/project/tdfs4ds/'>here.</a></p>

<b style = 'font-size:18px;font-family:Arial;color:#00233C'>Filters:</b>
<li style = 'font-size:16px;font-family:Arial;color:#00233C'><b>Industry:</b> Finance</li>
<li style = 'font-size:16px;font-family:Arial;color:#00233C'><b>Functionality:</b> Machine Learning</li> 
<li style = 'font-size:16px;font-family:Arial;color:#00233C'><b>Use Case:</b> Anomaly Detection</li></p>
<b style = 'font-size:18px;font-family:Arial;color:#00233C'>Related Resources:</b>
<li style = 'font-size:16px;font-family:Arial;color:#00233C'><a href = 'https://www.teradata.com/Blogs/Hyper-scale-time-series-forecasting-done-right'>Hyper-scale time series forecasting done right</a> </li>
<li style = 'font-size:16px;font-family:Arial;color:#00233C'><a href = 'https://www.teradata.com/Blogs/NPS-is-a-metric-not-the-goal'>In the fight to improve customer experience, NPS is a metric, not the goal</a></li>
<li style = 'font-size:16px;font-family:Arial;color:#00233C'><a href = 'https://www.teradata.com/Blogs/Customer-360-Analytics-What-Lies-Ahead'>Customer 360 Analytics, What Lies Ahead?</a></li>



<footer style="padding-bottom:35px; background:#f9f9f9; border-bottom:3px solid #00233C">
    <div style="float:left;margin-top:14px">ClearScape Analytics™</div>
    <div style="float:right;">
        <div style="float:left; margin-top:14px">
            Copyright © Teradata Corporation - 2024. All Rights Reserved.
        </div>
    </div>
</footer>